In [3]:
import numpy as np
import core
import pylab as plt
import nibabel as nib
import os
from skimage import filters
import scipy.ndimage as ndi
from scipy.ndimage.morphology import binary_erosion as be
from scipy.ndimage.morphology import binary_fill_holes as bfh
from scipy.ndimage.morphology import binary_dilation as bd
from scipy.ndimage.morphology import binary_opening as bo
from scipy.ndimage.morphology import binary_closing as bc
from scipy.ndimage.morphology import grey_dilation as gd
import time
from skimage.segmentation import find_boundaries as boundaries
import skimage
import SimpleITK as sitk

In [4]:
import nipype.interfaces.fsl as fsl

def extract(thresholded):
    img = nib.Nifti1Image(thresholded, np.eye(4))
    nib.save(img, 'temp.nii.gz')
    img = nib.load('temp.nii.gz')
    mybet = fsl.BET()
    result = mybet.run(in_file='temp.nii.gz', out_file='BET-temp.nii.gz', frac=0.1)
    img = nib.load('BET-temp.nii.gz')
    os.remove('BET-temp.nii.gz')
    os.remove('temp.nii.gz')
    data = img.get_data()
    return data

In [48]:
def saveMask(array, path, name):
    mask = array.astype(np.float64)
    mask = np.swapaxes(mask, 0, 1)
    img = nib.Nifti1Image(mask, affine)
    savePath = os.path.join(os.path.split(path)[0], '{}.nii.gz'.format(name))
    nib.save(img, savePath)

In [14]:
def saveMask(path, maskArray, saveName):
    maskArraySwapped = np.swapaxes(maskArray, 0, 2)
    img = sitk.GetImageFromArray(maskArraySwapped)
    img.CopyInformation(image)
    if path.endswith('.nii') or path.endswith('.nii.gz') or path.endswith('nrrd'):
        pathHead = os.path.split(path)[0]
    elif path.endswith('.dcm'):
        pathHead = os.path.split(os.path.split(path)[0])[0]
    savePath = os.path.join(pathHead, saveName)
    sitk.WriteImage(img, savePath)


In [8]:
path = '/Volumes/SINEADUSB/DC-project/Pre-op.nii.gz'

In [15]:
saveMask(path, array, 'hi.nii.gz')

In [49]:
info = []
distance = []
planes = np.load('/Volumes/SINEADUSB/DC-project/code/planes-params.npy')

In [50]:
paths = ['/Volumes/SINEADUSB/Scans/ScanB/1.3.12.2.1107.5.1.4.51967.30000014110308064564000003547.dcm', 
        '/Volumes/SINEADUSB/Scans/ScanD/1.3.12.2.1107.5.1.4.95198.30000015040208110913600035431.dcm',
        '/Volumes/SINEADUSB/Scans/ScanE/1.3.12.2.1107.5.1.4.95198.30000015040208110913600035427.dcm',
        '/Volumes/SINEADUSB/Scans/ScanF/1.3.12.2.1107.5.1.4.95198.30000015061008243097400047435.dcm',
        '/Volumes/SINEADUSB/Scans/ScanG/1.3.12.2.1107.5.1.4.95198.30000015051507521609300034665.dcm',
        '/Volumes/SINEADUSB/Scans/ScanH/1.3.12.2.1107.5.1.4.95198.30000015072708211605400036461.dcm',
        '/Volumes/SINEADUSB/Scans/ScanI/1.3.12.2.1107.5.1.4.95198.30000015071109310409300075672.dcm',
        '/Volumes/SINEADUSB/Scans/ScanJ/1.2.392.200036.9116.2.6.1.48.1221600807.1432222434.802585.dcm',
        '/Volumes/SINEADUSB/Scans/ScanK/1.2.392.200036.9116.2.6.1.48.1221600807.1432222434.802585.dcm',
        '/Volumes/SINEADUSB/Scans/ScanL/1.2.392.200036.9116.2.6.1.48.1221600807.1432222434.802585.dcm',
        '/Volumes/SINEADUSB/Scans/ScanM/1.2.392.200036.9116.2.6.1.48.1221600807.1432222434.802585.dcm',
        '/Volumes/SINEADUSB/Scans/ScanN/1.2.392.200036.9116.2.6.1.48.1221600807.1432222434.802585.dcm',
        '/Volumes/SINEADUSB/Scans/ScanO/1.2.392.200036.9116.2.6.1.48.1221600807.1432222434.802585.dcm']

In [51]:
pixelspacing = []
csvinfo = []

In [13]:
if path.endswith('.nii') or path.endswith('.nii.gz') or path.endswith('nrrd'):
    image        = sitk.ReadImage(path)
    array        = np.swapaxes(sitk.GetArrayFromImage(image), 0, 2)
    pixelspacing = self.scan.image.GetSpacing()
    bone         = core.thresholdnp(array, 1100, 1500)
    softtissue   = core.thresholdnp(array, 0, 80)

# elif self.scan.path.endswith('.dcm'):
#     pathHead               = os.path.split(self.scan.path)[0]
#     reader                 = sitk.ImageSeriesReader()
#     dicomNames             = reader.GetGDCMSeriesFileNames(pathHead)
#     reader.SetFileNames(dicomNames)
#     self.scan.image        = reader.Execute()
#     self.scan.array        = np.swapaxes(sitk.GetArrayFromImage(self.scan.image), 0, 2)
#     self.scan.pixelspacing = self.scan.image.GetSpacing()
#     self.scan.bone         = core.thresholdnp(self.scan.array, 900, 1500)
#     self.scan.softtissue   = core.thresholdnp(self.scan.array, 0, 80)

NameError: name 'self' is not defined

In [52]:
    # skin
    skin_mask = skin_mask = ndi.filters.gaussian_filter(array, 1)

    skin_mask = np.multiply(skin_mask, (array>-200).astype(int))
    array1 = np.multiply(skin_mask, (skin_mask>0.0).astype(int))
    array1 = np.multiply(array1, (array1>-200).astype(int))

    # skull
    skull_mask = array1>84.0
    array2 = np.multiply(array1, (array1<84.0).astype(int))

    # orbital
    orbital_mask = skull_mask

    for i in range(5):
        orbital_mask = bd(skull_mask)

    array3 = np.multiply(array2, (orbital_mask==False).astype(int))

    # cerebral ventricle
    array4 = np.multiply(array3, (array3>0.0).astype(int))

    # BET FSL
    array5 = extract(array4)
    brain_mask = np.multiply(bc(bfh(bd(array5))).astype(int), array5)

    # get ventricles
    ventr_mask1 = np.multiply((brain_mask>0).astype(int), (brain_mask<22).astype(int))
    ventr_mask2 = bd(ventr_mask1)
    ventr_mask3 = be(ventr_mask2)
    ventr_mask4 = bd(ventr_mask3)

    # Ventricle Analysis
    brainArea = []
    for i in range(brain_mask.shape[2]):
        brainArea.append(np.count_nonzero(brain_mask[:,:,i]))
    midsliceIndex = np.argmax(brainArea)
    
    import skimage

    ventr_loop = ventr_mask3
    
    for ind in range(1):
        labels = skimage.measure.label(ventr_loop, connectivity=1)
        props = skimage.measure.regionprops(labels)
        v = [p.area for p in props]
        x = 1
        ind = v.index(max(v))
        vcoords = props[ind].coords
        while abs(np.mean(vcoords.transpose()[2])-midsliceIndex)>array.shape[2]/10:
            ind = np.argsort(v)[-x]
            vcoords = props[ind].coords
            x = x+1
        ventr_loop = np.zeros((array.shape))
        for i in range(len(vcoords)):
            a,b,c = vcoords[i]
            ventr_loop[a,b,c]=1
        ventr_loop = bd(be(ventr_loop))


    for i in range(3):
        labels = skimage.measure.label(ventr_loop, connectivity=1)
        props = skimage.measure.regionprops(labels)
        v = [p.area for p in props]
        ind = v.index(max(v))
        vcoords = props[ind].coords
        ventr_loop = np.zeros((array.shape))
        for i in range(len(vcoords)):
            a,b,c = vcoords[i]
            ventr_loop[a,b,c]=1
        ventr_loop = bd(be(ventr_loop)) 
        
    labels = skimage.measure.label(ventr_loop, connectivity=1)
    props = skimage.measure.regionprops(labels)
    v = [p.area for p in props]
    ind = v.index(max(v))
    vcoords = props[ind].coords
    ventr_loop = np.zeros((array.shape))
    for i in range(len(vcoords)):
        a,b,c = vcoords[i]
        ventr_loop[a,b,c]=1
        
    ventr_loop = bd(be(ventr_loop))

    v_mask = bfh(ventr_loop)
    
    saveMask(v_mask, path, 'resultsvmask')

    # get haematoma

    h_mask = (array5>50.0).astype(float)*1000
    array6 = np.multiply(brain_mask, h_mask)
    array7 = be(array6)
    array8 = bc((array7))
    array9 = bfh((array8))
    array10 = bd(bd((array8)))
    array11 = bfh(bd(array10))
    import skimage

    h_loop = array11
        
    for i in range(4):
        labels = skimage.measure.label(h_loop, connectivity=1)
        props = skimage.measure.regionprops(labels)
        h = [p.area for p in props]
        ind = h.index(max(h))
        hcoords = props[ind].coords
        h_loop = np.zeros((array.shape))
        for i in range(len(hcoords)):
            a,b,c = hcoords[i]
            h_loop[a,b,c]=1
        h_loop = be(h_loop)

    labels = skimage.measure.label(h_loop, connectivity=1)
    props = skimage.measure.regionprops(labels)
    h = [p.area for p in props]
    ind = h.index(max(h))
    hcoords = props[ind].coords
    h_loop = np.zeros((array.shape))
    for i in range(len(hcoords)):
        a,b,c = hcoords[i]
        h_loop[a,b,c]=1
    h_loop = bd(h_loop)

    h_mask = bfh(h_loop)

    saveMask(h_mask, path, 'resultshmask')

    # volumes analysis
    plane = planes[pathnumber]
    a,b,c,d = plane[1:].astype(np.float64)

    normal = [a,b,c]

    i=0
    crossShape = array[:,:,i].shape
    mask = np.zeros(array.shape)

    if abs(normal[1])>abs(normal[0]):
        print 0
        for i in range(bone.shape[2]):
            z = i
            maski = np.fromfunction(lambda x,y: y > (d-c*i-a*x)/b, crossShape)
            mask[:,:,i] = maski

    if abs(normal[1])<abs(normal[0]):
        print 1
        for i in range(bone.shape[2]):
            z = i
            maski = np.fromfunction(lambda x,y: x > ((d-z*c-y*b)/a), crossShape)
            mask[:,:,i] = maski

    midplane_split = np.swapaxes(mask, 0, 1)

    v_mask = (v_mask).astype(int)
    h_mask = (h_mask).astype(int)

    left_h = np.multiply(h_mask, midplane_split==0)
    right_h = np.multiply(h_mask, midplane_split==1)
    
    saveMask(left_h, path, 'resultslefth')
    saveMask(right_h, path, 'resultsrighth')

    left_v = np.multiply(v_mask, midplane_split==0)
    right_v = np.multiply(v_mask, midplane_split==1)
    
    saveMask(left_v, path, 'resultsleftv')
    saveMask(right_v, path, 'resultsrightv')

    brain = np.multiply(brain_mask, (h_mask==0).astype(int)) # no haematoma
    brain_mask = np.multiply(brain_mask, (h_mask==0).astype(int)) # no haematoma
    brain_mask = (brain_mask!=0).astype(int)
    
    saveMask(brain, path, 'resultsbrain')
    saveMask(brain_mask, path, 'resultsbrainmask')

    left_brain = np.multiply(brain, midplane_split==0)
    right_brain = np.multiply(brain, midplane_split==1)
    

    left_brain_mask = np.multiply(brain_mask, midplane_split==0)
    right_brain_mask = np.multiply(brain_mask, midplane_split==1)

    saveMask(left_brain_mask, path, 'resultsleftbrainmask')
    saveMask(right_brain_mask, path, 'resultsrightbrainmask')
    
    # check that there is ventricle at the midsliceIndex

    if (np.nonzero(v_mask[:,:,midsliceIndex])[0].size == 0):
        print 'using vArea for vcentroid'
        vArea = []
        for i in range(brain_mask.shape[2]):
            vArea.append(np.count_nonzero(v_mask[:,:,i]))
        midsliceIndex = np.argmax(vArea)

    i = midsliceIndex
    w3 = i

    nonzeros = np.nonzero(np.nan_to_num(v_mask[:,:,i]))
    w2, w1 = np.mean(nonzeros[0]), np.mean(nonzeros[1])

    left_h_vol = (left_h[left_h!=0]).size*np.prod(ConstPixelSpacing)
    right_h_vol = (right_h[right_h!=0]).size*np.prod(ConstPixelSpacing)
    left_brain_vol = (left_brain[left_brain!=0]).size*np.prod(ConstPixelSpacing)
    right_brain_vol = (right_brain[right_brain!=0]).size*np.prod(ConstPixelSpacing)
    left_ventr_vol = (left_v[left_v!=0]).size*np.prod(ConstPixelSpacing)
    right_ventr_vol = (right_v[right_v!=0]).size*np.prod(ConstPixelSpacing)

    dirname = os.path.split(os.path.dirname(path))[1]
    
    normal = np.array([a,b,c])
    vcentroid = np.array([w1, w2, w3])
    pointOnPlane = np.array([vcentroid[0], (d-c*array.shape[2]/2-a*vcentroid[0])/b, array.shape[2]/2])
    n = np.multiply(normal, ConstPixelSpacing)
    n = n/np.linalg.norm(n)
    dist = np.dot(n, np.multiply(vcentroid-pointOnPlane, ConstPixelSpacing))
    print dist
    csvinfo.append((dirname, w1, w2, w3, left_h_vol, right_h_vol, left_brain_vol, right_brain_vol, left_ventr_vol, right_ventr_vol, dist, ConstPixelSpacing))
    
#     print info[pathnumber]

iteration number 0
1
12.8068889257
iteration number 1
1
-17.8991210223
iteration number 2
1
14.7265459096
iteration number 3
1
-19.2172982719
iteration number 4
1
24.3285598779
iteration number 5
1
-23.1269416321
iteration number 6
1
using vArea for vcentroid
-51.1768346706
iteration number 7
1
-26.0149237512
iteration number 8
1
10.2081555636
iteration number 9
1
30.0759716078
iteration number 10
1
20.845547136
iteration number 11
1
-2.62843573979
iteration number 12
1
12.3114696847


In [53]:

# %matplotlib nbagg
# pointOnPlane = np.array([vcentroid[0], (d-c*array.shape[2]/2-a*vcentroid[0])/b, array.shape[2]/2])
# plt.imshow(array[:,:,int(pointOnPlane[2])])
# plt.scatter(pointOnPlane[0], pointOnPlane[1], marker=10, c='red')
# plt.scatter(vcentroid[0], vcentroid[1], marker=10, c='yellow')
# plt.ion()

In [54]:
import pandas as pd

In [55]:
dfCsvInfo = pd.DataFrame(csvinfo)
dfCsvInfo.columns = ('Scan Name', 'w1', 'w2', 'w3', 'left haematoma volume', 'right haematoma volume', 'left brain volume', 'right brain volume', 'left ventricle volume', 'right ventricle volume', 'distance from ventricle centroid to midplane', 'pixel spacing')

In [56]:
dfCsvInfo

,Scan Name,w1,w2,w3,left haematoma volume,right haematoma volume,left brain volume,right brain volume,left ventricle volume,right ventricle volume,distance from ventricle centroid to midplane,pixel spacing
0,ScanB,303.647609,283.887301,59,23874.388252,0.000000,794194.415874,7.676752e+05,17797.815715,31133.674266,12.806889,"(0.453125, 0.453125, 1.50359663866)"
1,ScanD,194.536723,286.188324,157,10002.605908,101837.373126,732101.636841,5.352444e+05,10183.900909,0.000000,-17.899121,"(0.4609375, 0.4609375, 0.7)"
2,ScanE,283.019814,257.984848,137,49884.954332,6365.439308,614627.870224,6.792914e+05,0.000000,5325.709460,14.726546,"(0.45703125, 0.45703125, 0.7)"
3,ScanF,241.922081,281.443237,183,64033.288184,108746.652075,657551.111328,5.383620e+05,33542.700830,371.824414,-19.217298,"(0.484375, 0.484375, 0.7)"
4,ScanG,200.783485,279.353853,94,79228.800812,122643.625183,611043.242493,4.761683e+05,49977.988739,125.106201,24.328560,"(0.4296875, 0.4296875, 0.7)"
5,ScanH,189.490723,286.805822,58,31.479435,82578.748986,630354.196958,6.124826e+05,38771.659482,0.000000,-23.126942,"(0.449219, 0.449219, 1.26825210084)"
6,ScanI,160.505800,384.178654,174,16055.167102,84275.020297,705978.839691,5.737771e+05,454.251259,0.000000,-51.176835,"(0.431640625, 0.431640625, 0.7)"
7,ScanJ,258.167134,341.056085,76,0.000000,42607.251456,656958.181376,4.974210e+05,14709.997568,0.000000,-26.014924,"(0.448, 0.448, 1.0)"
8,ScanK,289.280189,265.974528,189,64983.029248,15097.159039,748673.397961,8.530416e+05,0.000000,4530.739063,10.208156,"(0.4609375, 0.4609375, 0.7)"
9,ScanL,281.564140,238.215743,13,0.000000,20.517902,18451.628758,2.827174e+05,0.000000,9426.165659,30.075972,"(0.488281, 0.488281, 5.06225925926)"


In [57]:
dfCsvInfo.to_csv('resultssymmetrybatch.csv')